In [22]:
import pandas as pd
import subprocess
import json
import sys

pd.set_option('max_colwidth',250)
pd.set_option('display.max_rows',90)
# sys.setdefaultencoding('utf-8')

AttributeError: module 'sys' has no attribute 'setdefaultencoding'

In [2]:
def print_tags(element, indent=0):
        print("  " * indent + element.tag)
        for child in element:
            print_tags(child, indent + 1)

def tabelaParlamentar(parlamentares):
    tabela = []
    for parlamentar in parlamentares:
        treeParlamentar = etree.fromstring(etree.tostring(parlamentar, encoding="unicode"))
        try:
            partido = treeParlamentar.xpath('//IdentificacaoParlamentar//SiglaPartidoParlamentar/text()')[0]
        except:
            partido = "N/I"

        try:
            codigoBloco = treeParlamentar.xpath('//IdentificacaoParlamentar//Bloco//CodigoBloco/text()')[0]
            NomeBloco = treeParlamentar.xpath('//IdentificacaoParlamentar//Bloco//NomeBloco/text()')[0]
            NomeApelido = treeParlamentar.xpath('//IdentificacaoParlamentar//Bloco//NomeApelido/text()')[0]
            DataCriacao = treeParlamentar.xpath('//IdentificacaoParlamentar//Bloco//DataCriacao/text()')[0],
        except:
            codigoBloco = "N/I"
            NomeBloco = "N/I"
            NomeApelido = "N/I"
            DataCriacao = "N/I"

        tabela.append([
            treeParlamentar.xpath('//IdentificacaoParlamentar//CodigoParlamentar/text()')[0],
            treeParlamentar.xpath('//IdentificacaoParlamentar//CodigoPublicoNaLegAtual/text()')[0],
            treeParlamentar.xpath('//IdentificacaoParlamentar//NomeParlamentar/text()')[0],
            treeParlamentar.xpath('//IdentificacaoParlamentar//NomeCompletoParlamentar/text()')[0],
            treeParlamentar.xpath('//IdentificacaoParlamentar//SexoParlamentar/text()')[0],
            treeParlamentar.xpath('//IdentificacaoParlamentar//FormaTratamento/text()')[0],
            treeParlamentar.xpath('//IdentificacaoParlamentar//UrlFotoParlamentar/text()')[0],
            treeParlamentar.xpath('//IdentificacaoParlamentar//UrlPaginaParlamentar/text()')[0],
            treeParlamentar.xpath('//IdentificacaoParlamentar//EmailParlamentar/text()')[0],
            partido,
            treeParlamentar.xpath('//IdentificacaoParlamentar//UfParlamentar/text()')[0],
            codigoBloco,
            NomeBloco,
            NomeApelido,
            DataCriacao,
            treeParlamentar.xpath('//IdentificacaoParlamentar//MembroMesa/text()')[0],
            treeParlamentar.xpath('//IdentificacaoParlamentar//MembroLideranca/text()')[0],
        ])
    return tabela

def api(api_url):
    response = requests.get(api_url)
    if response.status_code == 200:
        return (response.content)
    else:
        print(f"Error: {response.status_code}")

def retornarTree(api_url):
    string = api(api_url)
    tree = etree.fromstring(string)
    root = tree.tag
    return [tree,root]

def parlamentarMain():
    api_url = 'https://legis.senado.leg.br/dadosabertos/senador/lista/atual'
    tree = retornarTree(api_url)
    root = tree[1]
    tree = tree[0]
    parlamentares = tree.xpath('//ListaParlamentarEmExercicio//Parlamentares//Parlamentar')
    colunas = ["CodigoParlamentar","CodigoPublicoNaLegAtual","NomeParlamentar","NomeCompletoParlamentar","SexoParlamentar","FormaTratamento","UrlFotoParlamentar","UrlPaginaParlamentar","EmailParlamentar","SiglaPartidoParlamentar","UfParlamentar","CodigoBloco","NomeBloco","NomeApelido","DataCriacao","MembroMesa","MembroLideranca",]
    tabelaParlamentares = tabelaParlamentar(parlamentares)
    salvarArquivo(tabelaParlamentares,"parlamentares",colunas)
    return pd.DataFrame(tabelaParlamentares, columns=colunas)

def salvarArquivo(lista,nome,colunas):
    dados = pd.DataFrame(lista, columns=colunas)
    dados.to_csv("arquivos/"+nome+".csv")

In [3]:
parlamentares = parlamentarMain()

In [21]:
def listaAutoriasParlamentares(Autorias,CodigoParlamentar):
    tabela = []
    for Autoria in Autorias:
        treeAutoria = etree.fromstring(etree.tostring(Autoria, encoding="unicode"))
        try:
            IndicadorOutrosAutores = treeAutoria.xpath('//Autoria//IndicadorOutrosAutores/text()')[0]
        except:
            IndicadorOutrosAutores = "N/I"
        try:
            Ementa = treeAutoria.xpath('//Materia//Ementa/text()')[0]
        except:
            Ementa = "N/I"
        tabela.append([
            CodigoParlamentar,
            treeAutoria.xpath('//Materia//Codigo/text()')[0],
            treeAutoria.xpath('//Materia//IdentificacaoProcesso/text()')[0],
            treeAutoria.xpath('//Materia//DescricaoIdentificacao/text()')[0],
            treeAutoria.xpath('//Materia//Sigla/text()')[0],
            treeAutoria.xpath('//Materia//Numero/text()')[0],
            treeAutoria.xpath('//Materia//Ano/text()')[0],
            Ementa,
            treeAutoria.xpath('//Materia//Data/text()')[0],
            IndicadorOutrosAutores
        ])
    return tabela

def AutoriasPorParlamentar(CodigoParlamentar):
    api_url = 'https://legis.senado.leg.br/dadosabertos/senador/'+CodigoParlamentar+'/autorias'
    tree = retornarTree(api_url)
    root = tree[1]
    tree = tree[0]
    Autorias = tree.xpath('//MateriasAutoriaParlamentar//Parlamentar//Autorias//Autoria')
    listaAutorias = listaAutoriasParlamentares(Autorias,CodigoParlamentar)
    return listaAutorias

In [38]:
def autoriasMain():
    autoriasTEMP = []
    for i,r in parlamentares.iterrows():
        autoriasTEMP.append(AutoriasPorParlamentar(r[0]))
    autorias = []
    for i in range(len(autoriasTEMP)):
        for j in range(len(autoriasTEMP[i])):
            autorias.append(autoriasTEMP[i][j])
    autoriasTEMP = None
    colunas = ["CodigoParlamentar","CodigoMateria","IdentificacaoProcesso","DescricaoIdentificacao","Sigla","Numero","Ano","Ementa","Data","IndicadorOutrosAutores",]
    autorias = pd.DataFrame(autorias,columns=colunas)
    salvarArquivo(autorias,"autorias",colunas)
    return autorias

In [51]:
comando = 'curl https://legis.senado.leg.br/dadosabertos/arquivos/ListaParlamentarEmExercicio.json "Content-Type: text/xml; charset=utf-8-sig"'
resultado = subprocess.run(comando, shell=True, text=True, capture_output=True)
dados = json.loads(resultado.stdout)
dados = pd.DataFrame(dados)
dados = (dados.loc["Parlamentares"]["ListaParlamentarEmExercicio"])
dados = pd.DataFrame(dados["Parlamentar"])
saida = pd.DataFrame()
for index,row in dados.iterrows():
    IdentificacaoParlamentar = pd.DataFrame(row["IdentificacaoParlamentar"])
    Mandato = pd.DataFrame(row["Mandato"])
    concatenar = pd.concat([IdentificacaoParlamentar,Mandato])
    saida = pd.concat([saida,concatenar])
# dados = pd.json_normalize(dados)
# dados = resultado.stdout
# dados = pd.read_json(dados, orient='index')

In [52]:
IdentificacaoParlamentar = dados["IdentificacaoParlamentar"]
Mandato = dados["Mandato"]

In [53]:
dadosTMP = pd.DataFrame() 
for i in range(0,81):
    tmpIP = pd.DataFrame(IdentificacaoParlamentar[i])
#     mpM = pd.DataFrame(Mandato[i])
    tmpIP = tmpIP.reset_index()
#     mpM = tmpM.reset_index()
    tmpTel = tmpIP.iloc[0]["Telefones"]
    try:
        tmpTels = pd.DataFrame(tmpTel, index=["NumeroTelefone","OrdemPublicacao","IndicadorFax"])
    except:
        tmpTels = pd.DataFrame(tmpTel[0], index=["NumeroTelefone","OrdemPublicacao","IndicadorFax"])
        
    tmpIP.loc[0, "Telefones"] = tmpTels.loc["NumeroTelefone", "NumeroTelefone"]
    tmpIP.loc[0, "CodigoBloco"] = tmpIP.loc[1, "Bloco"]
    tmpIP.loc[0, "NomeBloco"] = tmpIP.loc[2, "Bloco"]
    tmpIP.loc[0, "NomeApelido"] = tmpIP.loc[3, "Bloco"]
    tmpIP.loc[0, "DataCriacao"] = tmpIP.loc[4, "Bloco"]
    tmpIP.drop([1,2,3,4], inplace=True)
    #mpIP.join(tmpM)
    dadosTMP = pd.concat([tmpIP,dadosTMP])
    dados = dadosTMP.drop(columns="index").reset_index().drop(columns="index")

C:\Users\kristyan.silva\AppData\Local\Temp\ipykernel_16028\2143137496.py:14: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '336' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  tmpIP.loc[0, "CodigoBloco"] = tmpIP.loc[1, "Bloco"]
C:\Users\kristyan.silva\AppData\Local\Temp\ipykernel_16028\2143137496.py:15: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'Bloco Parlamentar Democracia' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  tmpIP.loc[0, "NomeBloco"] = tmpIP.loc[2, "Bloco"]
C:\Users\kristyan.silva\AppData\Local\Temp\ipykernel_16028\2143137496.py:16: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'BLDEM' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.


In [54]:
autorias

,CodigoParlamentar,CodigoMateria,IdentificacaoProcesso,DescricaoIdentificacao,Sigla,Numero,Ano,Ementa,Data,IndicadorOutrosAutores
0,5672,155883,8368927,RQS 65/2023,RQS,00065,2023,Requer o desarquivamento do PLS nº 166/2018.,2023-02-14,N/I
1,5672,157431,8450980,RQS 467/2023,RQS,00467,2023,"Requeremos calendário especial para a Proposta de Emenda à Constituição nº 162, de 2019, que ""altera a redação do inciso VIII-A do caput do art. 93 da Constituição Federal, para permitir a permuta entre juízes de direito vinculados a diferentes t...",2023-05-16,N/I
2,5672,161949,8617362,RQS 46/2024,RQS,00046,2024,Requer a realização de Sessão Especial destinada a celebrar o Dia da Bíblia.,2024-02-06,N/I
3,5672,161956,8617716,RQS 50/2024,RQS,00050,2024,Requer a realização de Sessão Especial destinada a celebrar o Dia do Oftalmologista.,2024-02-06,N/I
4,5672,161954,8617712,RQS 48/2024,RQS,00048,2024,Requer a realização de Sessão Especial destinada a celebrar o aniversário de Roraima.,2024-02-06,N/I
...,...,...,...,...,...,...,...,...,...,...
42512,3806,135547,7723599,RQS 97/2019,RQS,00097,2019,"Realização de sessão especial, no dia 19 de agosto destinada a comemorar o dia do Maçom Brasileiro.",2019-03-07,N/I
42513,3806,135594,7725468,PEC 17/2019,PEC,00017,2019,"Acrescenta o inciso XII-A, ao art. 5º, e o inciso XXX, ao art. 22, da Constituição Federal para incluir a proteção de dados pessoais entre os direitos fundamentais do cidadão e fixar a competência privativa da União para legislar sobre a matéria.",2019-03-12,N/I
42514,3806,135120,7711484,PEC 1/2019,PEC,00001,2019,Acrescenta ao art. 57 o §4º-A para dispor sobre o voto aberto na eleição das mesas no congresso.,2019-02-07,N/I
42515,3806,135124,7713383,RQS 9/2019,RQS,00009,2019,Desarquivamento da PEC 29/2015.,2019-02-06,N/I


In [55]:
dados.to_csv("../Dados/Senadores.csv",encoding='utf-8-sig')